# ML Assignment 2: Optimization

## Overview

In developing this script, I referenced
 Hayes, G. (2019). mlrose: Machine Learning, Randomized Optimization and SEarch package for Python. https://github.com/gkhayes/mlrose 
 and also https://github.com/hiive/mlrose

mlrose is a Python package for applying some of the most common randomized optimization and search algorithms to a range of different optimization problems, over both discrete- and continuous-valued parameter spaces. This notebook contains the examples used in the mlrose tutorial.

### Import Libraries

In [1]:
import time
import threading

import mlrose_hiive as mlrose
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

### Problem 1:  Fill the Knapsack!

In [2]:
class Runner:
    def __init__(self,n, problem_name, algs, alg_names):
        self.n = n
        self.name = problem_name
        fig1, ax1 = plt.subplots()
#         ax1.title = "Score Per Iteration"
        ax1.set_ylabel("Fitness score")
        ax1.set_xlabel("Iteration")
        self.fig1 = fig1
        self.ax1 = ax1
        
        fig2, ax2 = plt.subplots()
#         ax2.title = "Evals Per Iteration"
        ax2.set_ylabel("Fitness Evals")
        ax2.set_xlabel("Iterations")
        self.fig2 = fig2
        self.ax2 = ax2
        
        self.algs = algs
        self.names = alg_names
        self.times = []
        self.best_fitnesses = []
        
    def run(self):
        for algorithm, name in zip(self.algs,self.names):
            print(f'starting {name}....')
            tic = time.perf_counter()
            best_state, best_fitness, fitness_curve = algorithm()
            toc = time.perf_counter()
#             print(f"{name} completed in {toc - tic:0.4f} seconds")
#             print(f'{name} found best fitness/value: ', best_fitness, '\n')
            
            self.times.append(toc-tic)
            self.best_fitnesses.append(best_fitness)
            self.plot_curve(fitness_curve, f'{name}')
        
        self.fig1.legend(loc=4)
        self.fig2.legend(loc=4)
        self.fig1.savefig(f'charts/{self.name}_{n}_items.png', bbox_inches='tight')
        self.fig2.savefig(f'charts/{self.name}_{n}_items_evals.png', bbox_inches='tight')
        self.fig1.clf()
        self.fig2.clf()
    def print_stats(self):
        for name, time, best_fitness in zip(self.names, self.times, self.best_fitnesses):
            print(f'{name} had \n\tTime: {time:0.4f} seconds\n\tFitness Score: {best_fitness}')
    def plot_curve(self, fitness_curve, name):
        self.ax1.plot(range(0,len(fitness_curve)), fitness_curve[:,0], label=f'{name}')
        self.ax2.plot(range(0,len(fitness_curve)), fitness_curve[:,1], label=f'{name}')
    


In [3]:
# Initialize fitness function object using pre-defined class
'''
Fitness function for Knapsack optimization problem. Given a set of n
items, where item i has known weight :math:`w_{i}` and known value
:math:`v_{i}`; and maximum knapsack capacity, :math:`W`, the Knapsack
fitness function evaluates the fitness of a state vector
:math:`x = [x_{0}, x_{1}, \ldots, x_{n-1}]` as:
'''
# https://en.wikipedia.org/wiki/Knapsack_problem
# We're trying to get to the highest value possible, without going over our weight limit

ns = [50,100, 200, 400, 800, 1000,1500] # items in our knapsack
weights = []
values = []
fitnesses = []
init_states = []
for n in ns:
    weight = (np.random.randint(1,50,size=(n)))
    value = (np.random.randint(1,50,size=(n)))
    init_state = np.random.randint(0,1,size=(n))
    init_states.append(init_state)
    max_weight_pct = 0.6 # so we can only hold 60% of our total weight for items we're trying to fit
    fitnesses.append(mlrose.Knapsack(weight, value, max_weight_pct))
    weights.append(weight)
    values.append(value)

# basically have a seed state so we can reproduce
random_state = 5
fitness_values= dict()
times = dict()
alg_names = ['random_hill_climbing','simulated_annealing','genetic_algorithm','mimic']
for name in alg_names:
    fitness_values[name] = []
    times[name] = []
for weight, value, fitness, n, init_state in zip(weights, values, fitnesses, ns, init_states):
    print('=====================')
    print(f'starting n = {n}...')
    algs = []
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    schedule = mlrose.GeomDecay(.01,0.0001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.simulated_annealing(problem, schedule=schedule, curve=True, random_state = random_state))
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, random_state=random_state))
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
 

    thing = Runner(n, 'knapsack', algs, alg_names)
    thing.run()
    thing.print_stats()
    
    for index, name in enumerate(alg_names):
        fitness_values[name].append(thing.best_fitnesses[index])
        times[name].append(thing.times[index])
    
fig1, ax1 = plt.subplots()
ax1.set_ylabel("Fitness score")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, fitness_values[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/knapsack_problem_size.png', bbox_inches='tight')
fig1.clf()

fig1, ax1 = plt.subplots()
ax1.set_ylabel("Seconds to Convergence")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, times[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/knapsack_times_problem_size.png', bbox_inches='tight')
fig1.clf()

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0009 seconds
	Fitness Score: 691.0
simulated_annealing had 
	Time: 0.0011 seconds
	Fitness Score: 756.0
genetic_algorithm had 
	Time: 0.5807 seconds
	Fitness Score: 1188.0
mimic had 
	Time: 0.2196 seconds
	Fitness Score: 1174.0
starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0009 seconds
	Fitness Score: 1596.0
simulated_annealing had 
	Time: 0.0011 seconds
	Fitness Score: 1627.0
genetic_algorithm had 
	Time: 0.9303 seconds
	Fitness Score: 2213.0
mimic had 
	Time: 0.8448 seconds
	Fitness Score: 2181.0
starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0018 seconds
	Fitness Score

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Part 2: Six Peaks 

In [4]:
# Initialize fitness function object using pre-defined class

ns = [50,100,200, 300,400, 500,600,700,800] # items in our knapsack

fitnesses = []
for n in ns:
    fitnesses.append(mlrose.SixPeaks(t_pct=0.20))


# basically have a seed state so we can reproduce


# basically have a seed state so we can reproduce
random_state = 5

fitness_values= dict()
times = dict()
alg_names = ['random_hill_climbing','simulated_annealing','genetic_algorithm','mimic']
for name in alg_names:
    fitness_values[name] = []
    times[name] = []
    
for fitness, n in zip(fitnesses, ns):
    print('=====================')
    print(f'starting n = {n}...')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs = []
    
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))
    schedule = mlrose.GeomDecay(.01,0.001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.simulated_annealing(problem,schedule=schedule, curve=True, random_state = random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, random_state=random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    
    thing = Runner(n, 'six_peaks', algs, alg_names)
    thing.run()
    thing.print_stats()
    for index, name in enumerate(alg_names):
        fitness_values[name].append(thing.best_fitnesses[index])
        times[name].append(thing.times[index])
    
fig1, ax1 = plt.subplots()
ax1.set_ylabel("Fitness score")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, fitness_values[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/six_peaks_problem_size.png', bbox_inches='tight')
fig1.clf()

fig1, ax1 = plt.subplots()
ax1.set_ylabel("Seconds to Convergence")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, times[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/six_peaks_times_problem_size.png', bbox_inches='tight')
fig1.clf()

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0005 seconds
	Fitness Score: 6.0
simulated_annealing had 
	Time: 0.0077 seconds
	Fitness Score: 76.0
genetic_algorithm had 
	Time: 0.7112 seconds
	Fitness Score: 83.0
mimic had 
	Time: 0.1949 seconds
	Fitness Score: 16.0
starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0002 seconds
	Fitness Score: 1.0
simulated_annealing had 
	Time: 0.0959 seconds
	Fitness Score: 63.0
genetic_algorithm had 
	Time: 0.8143 seconds
	Fitness Score: 136.0
mimic had 
	Time: 0.6473 seconds
	Fitness Score: 22.0
starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0003 seconds
	Fitness Score: 4.0
simulated

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Example 3: Queens

In [5]:
ns = [50,100,200,400,500] # items in our knapsack

fitnesses = []
init_coords = []
for n in ns:
    # Define alternative N-Queens fitness function for maximization problem
    # This code was ripped off from https://mlrose.readthedocs.io/en/stable/source/tutorial1.html#solving-optimization-problems-with-mlrose
    def queens_max(state):
       # Initialize counter
        fitness_cnt = 0
          # For all pairs of queens
        for i in range(len(state) - 1):
            for j in range(i + 1, len(state)):
                # Check for horizontal, diagonal-up and diagonal-down attacks
                if (state[j] != state[i]) and (state[j] != state[i] + (j - i)) and (state[j] != state[i] - (j - i)):
                   # If no attacks, then increment counter
                    fitness_cnt += 1
        return fitness_cnt

    # Initialize custom fitness function object
    fitness_cust = mlrose.CustomFitness(queens_max)
    fitnesses.append(fitness_cust)


# basically have a seed state so we can reproduce
random_state = 15
fitness_values= dict()
times = dict()
alg_names = ['random_hill_climbing','simulated_annealing','genetic_algorithm','mimic']
for name in alg_names:
    fitness_values[name] = []
    times[name] = []
    
for fitness, n in zip(fitnesses, ns):
    print('=====================')
    print(f'starting n = {n}...')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs = []
    
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    
    schedule = mlrose.GeomDecay(.01,0.001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.simulated_annealing(problem, curve=True, random_state = random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, max_iters=100, random_state=random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    
    thing = Runner(n, 'queens', algs, alg_names)
    thing.run()
    thing.print_stats()
    for index, name in enumerate(alg_names):
        fitness_values[name].append(thing.best_fitnesses[index])
        times[name].append(thing.times[index])
    print('\n')
    
fig1, ax1 = plt.subplots()
ax1.set_ylabel("Fitness score")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, fitness_values[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/queens_problem_size.png', bbox_inches='tight')
fig1.clf()

fig1, ax1 = plt.subplots()
ax1.set_ylabel("Seconds to Convergence")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, times[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/queens_times_problem_size.png', bbox_inches='tight')
fig1.clf()

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0373 seconds
	Fitness Score: 606.0
simulated_annealing had 
	Time: 0.0614 seconds
	Fitness Score: 607.0
genetic_algorithm had 
	Time: 2.6137 seconds
	Fitness Score: 612.0
mimic had 
	Time: 2.9115 seconds
	Fitness Score: 618.0


starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0920 seconds
	Fitness Score: 2450.0
simulated_annealing had 
	Time: 0.3310 seconds
	Fitness Score: 2473.0
genetic_algorithm had 
	Time: 20.3773 seconds
	Fitness Score: 2470.0
mimic had 
	Time: 15.2176 seconds
	Fitness Score: 2480.0


starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 1.3346 seconds
	Fitness S

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Example 6: Fitting a Neural Network to the Heart Dataset

In [6]:

df = pd.read_csv("data/heart.csv")
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]
 # https://www.kaggle.com/ronitf/heart-disease-uci


In [7]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 3)

In [8]:
# Normalize feature data
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# One hot encode target values
one_hot = OneHotEncoder()

y_train_hot = one_hot.fit_transform(np.array(y_train).reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(np.array(y_test).reshape(-1, 1)).todense()

In [19]:
class NNRunner:
    def __init__(self, alg_names):
        fig1, ax1 = plt.subplots()
        ax1.set_ylabel("Fitness score")
        ax1.set_xlabel("Iteration")
        self.fig1 = fig1
        self.ax1 = ax1
        
        fig2, ax2 = plt.subplots()
        ax2.set_ylabel("Fitness Evals")
        ax2.set_xlabel("Iterations")
        self.fig2 = fig2
        self.ax2 = ax2
        
        self.names = alg_names
        self.name = "NN"
        self.times = []
        self.scores = []
        
        
    # Copied from https://scikit-learn.org/stable/auto_examples/applications/plot_model_complexity_influence.html
    def benchmark_influence(self, conf):
        """
        Benchmark influence of `changing_param` on both MSE and latency.
        """
        prediction_times = []
        prediction_powers = []
        complexities = []
        for param_value in conf['changing_param_values']:
            conf['tuned_params'][conf['changing_param']] = param_value
            estimator = conf['estimator'](**conf['tuned_params'])

            print("Benchmarking %s" % estimator)
            estimator.fit(conf['data']['X_train'], conf['data']['y_train'])
            conf['postfit_hook'](estimator)
            complexity = conf['complexity_computer'](estimator)
            complexities.append(complexity)
            start_time = time.time()
            for _ in range(conf['n_samples']):
                y_pred = estimator.predict(conf['data']['X_test'])
            elapsed_time = (time.time() - start_time) / float(conf['n_samples'])
            prediction_times.append(elapsed_time)
            pred_score = conf['prediction_performance_computer'](
                conf['data']['y_test'], y_pred)
            prediction_powers.append(pred_score)
            print("Complexity: %d | %s: %.4f | Pred. Time: %fs\n" % (
                complexity, conf['prediction_performance_label'], pred_score,
                elapsed_time))
        return prediction_powers, prediction_times, complexities
    
    def count_nonzero_coefficients(self,estimator):
        a = estimator.coef_.toarray()
        return np.count_nonzero(a)

    
    def plot_influence(self,conf, mse_values, prediction_times, complexities):
        """
        Plot influence of model complexity on both accuracy and latency.
        """

        fig = plt.figure()
        fig.subplots_adjust(right=0.75)

        # first axes (prediction error)
        ax1 = fig.add_subplot(111)
        line1 = ax1.plot(complexities, mse_values, c='tab:blue', ls='-')[0]
        ax1.set_xlabel('Model Complexity (%s)' % conf['complexity_label'])
        y1_label = conf['prediction_performance_label']
        ax1.set_ylabel(y1_label)

        ax1.spines['left'].set_color(line1.get_color())
        ax1.yaxis.label.set_color(line1.get_color())
        ax1.tick_params(axis='y', colors=line1.get_color())

        # second axes (latency)
        ax2 = fig.add_subplot(111, sharex=ax1, frameon=False)
        line2 = ax2.plot(complexities, prediction_times, c='tab:orange', ls='-')[0]
        ax2.yaxis.tick_right()
        ax2.yaxis.set_label_position("right")
        y2_label = "Time (s)"
        ax2.set_ylabel(y2_label)
        ax1.spines['right'].set_color(line2.get_color())
        ax2.yaxis.label.set_color(line2.get_color())
        ax2.tick_params(axis='y', colors=line2.get_color())

        plt.legend((line1, line2), ("prediction error", "latency"),
                   loc='upper right')

        plt.title("Influence of varying '%s' on %s" % (conf['changing_param'],
                                                       conf['estimator'].__name__))


    def run(self):
        for name in self.names:
            print(f'starting {name}....')
            tic = time.perf_counter()
            print('===============================================')
            print(f'running NN weight optimization for {name}')
            # Initialize neural network object and fit object - attempt 1


            schedule = mlrose.GeomDecay(.01,0.0001) 
            nn = mlrose.NeuralNetwork(hidden_nodes = [2], activation ='relu', 
                                             algorithm =name, 
                                             max_iters = 100, bias = False, is_classifier = True, 
                                             learning_rate = 0.0001, early_stopping = True,
                                             schedule=schedule,
                                             restarts=10,
                                             curve=True,
                                             clip_max = 5, max_attempts = 1000, random_state = 5)
            _, axes = plt.subplots()
            axes.set_title("NN")
            axes.set_xlabel("Training examples")
            axes.set_ylabel("Score")
            train_sizes, train_scores, test_scores, fit_times, _ = \
                    learning_curve(nn, X_train_scaled, y_train_hot,return_times=True)

            train_scores_mean = np.mean(train_scores, axis=1)
            train_scores_std = np.std(train_scores, axis=1)
            test_scores_mean = np.mean(test_scores, axis=1)
            test_scores_std = np.std(test_scores, axis=1)
            fit_times_mean = np.mean(fit_times, axis=1)
            fit_times_std = np.std(fit_times, axis=1)

            # Plot learning curve
            axes.grid()
            axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                                 train_scores_mean + train_scores_std, alpha=0.1,
                                 color="r")
            axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                                 test_scores_mean + test_scores_std, alpha=0.1,
                                 color="g")
            axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                         label="Training score")
            axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                         label="Cross-validation score")
            axes.legend(loc="best")

            plt.savefig(f'charts/NN_{self.name}_{name}.png', bbox_inches='tight') 
            plt.clf()
            
            tic = time.perf_counter()
            nn.fit(X_train_scaled, y_train_hot)
            toc = time.perf_counter()
            y_test_pred = nn.predict(X_test_scaled)
            
            y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
            
            print("accuracy is:")
            print(y_test_accuracy)
            print(f'Took : {toc-tic:0.4f} seconds')
        
        self.fig1.legend(loc=4)
        self.fig2.legend(loc=4)
        self.fig1.savefig(f'charts/{self.name}_{n}_items.png', bbox_inches='tight')
        self.fig2.savefig(f'charts/{self.name}_{n}_items_evals.png', bbox_inches='tight')
        self.fig1.clf()
        self.fig2.clf()
    
    def plot_curve(self, fitness_curve, name):
        if len(fitness_curve.shape) == 2:
            self.ax1.plot(range(0,len(fitness_curve)), fitness_curve[:,0], label=f'{name}')
            self.ax2.plot(range(0,len(fitness_curve)), fitness_curve[:,1], label=f'{name}')
    


In [18]:
algorithms = ['random_hill_climb', 'simulated_annealing', 'genetic_alg','gradient_descent']
from sklearn.metrics import mean_squared_error
nn = NNRunner(algorithms)

classification_data =  {'X_train': X_train_scaled, 'X_test': X_test_scaled, 'y_train': y_train_hot, 'y_test': y_test_hot}
configurations = [
    {
        'estimator': mlrose.NeuralNetwork,
        'tuned_params': 
            {
                'learning_rate': 0.01 ,
                'hidden_nodes' : [2],
                'algorithm': 'genetic_alg',
                'activation': 'relu',
                'is_classifier': True,
                'early_stopping': True,
                'curve': True,
                'clip_max': 5,
                'max_attempts': 1000,
                'max_iters': 1000,
                'bias': False,
                'random_state': 5
            },
         'changing_param': 'hidden_nodes',
         'changing_param_values': [[2],[6],[8],[50],[100],[200]],
         'complexity_label': 'hidden nodes',
         'complexity_computer': lambda x: np.prod(x.hidden_nodes),
         'prediction_performance_computer': mean_squared_error,
         'prediction_performance_label': 'MSE',
         'postfit_hook': lambda x: x,
         'data': classification_data,
         'n_samples': len(X_test_scaled)
    },
        {
        'estimator': mlrose.NeuralNetwork,
        'tuned_params': 
            {
                'learning_rate': 0.01 ,
                'hidden_nodes' : [2],
                'algorithm': 'random_hill_climb',
                'activation': 'relu',
                'is_classifier': True,
                'early_stopping': True,
                'curve': True,
                'clip_max': 5,
                'max_attempts': 1000,
                'max_iters': 1000,
                'bias': False,
                'random_state': 5
            },
         'changing_param': 'hidden_nodes',
         'changing_param_values': [[2],[6],[8],[50],[100],[200]],
         'complexity_label': 'hidden nodes',
         'complexity_computer': lambda x: np.prod(x.hidden_nodes),
         'prediction_performance_computer': mean_squared_error,
         'prediction_performance_label': 'MSE',
         'postfit_hook': lambda x: x,
         'data': classification_data,
         'n_samples': len(X_test_scaled)
    },
    {
    'estimator': mlrose.NeuralNetwork,
    'tuned_params': 
        {
            'learning_rate': 0.01 ,
            'hidden_nodes' : [2],
            'algorithm': 'simulated_annealing',
            'activation': 'relu',
            'is_classifier': True,
            'early_stopping': True,
            'curve': True,
            'clip_max': 5,
            'max_attempts': 1000,
            'max_iters': 1000,
            'bias': False,
            'random_state': 5
        },
     'changing_param': 'hidden_nodes',
     'changing_param_values': [[2],[6],[8],[50],[100],[200]],
     'complexity_label': 'hidden nodes',
     'complexity_computer': lambda x: np.prod(x.hidden_nodes),
     'prediction_performance_computer': mean_squared_error,
     'prediction_performance_label': 'MSE',
     'postfit_hook': lambda x: x,
     'data': classification_data,
     'n_samples': len(X_test_scaled)
},
]



for conf in configurations:
    prediction_performances, prediction_times, complexities = nn.benchmark_influence(conf)
    nn.plot_influence(conf, prediction_performances, prediction_times, complexities)
    namey = conf['estimator'].__name__
    alg = conf['tuned_params']['algorithm']
    plt.savefig(f'charts/NN_{namey}_{alg}_complexity.png', bbox_inches='tight')
    plt.clf()

nn.run()

starting random_hill_climb....
running NN weight optimization for random_hill_climb
accuracy is:
0.5573770491803278
Took : 1.1436 seconds
starting simulated_annealing....
running NN weight optimization for simulated_annealing
accuracy is:
0.5573770491803278
Took : 0.1427 seconds
starting genetic_alg....
running NN weight optimization for genetic_alg


No handles with labels found to put in legend.
No handles with labels found to put in legend.


accuracy is:
0.8360655737704918
Took : 14.7721 seconds
starting gradient_descent....
running NN weight optimization for gradient_descent
accuracy is:
0.7213114754098361
Took : 0.1382 seconds


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>